In [1]:
%reset -sf

In [184]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [76]:
srr = """
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
""".strip()

In [77]:
# with open("/Users/hkmac/Downloads/input.txt") as f:
#     srr = f.read().strip()

In [78]:
# srr = srr.replace("$ ls", "")
# brr = srr.split("$")[1:]


In [79]:
from collections import defaultdict

In [80]:
stack = []
sizes = defaultdict(int)

for line in srr.split("\n"):
    if line == "$ cd ..":
        stack.pop()
        continue
    if line[:4] == "$ cd":
        stack.append(line.split()[-1])
        continue
    if line[:3] == "dir":
        continue
    if line[:4] == "$ ls":
        continue
    print(line)
    for i,x in enumerate(stack):
        sizes[tuple(stack[:i+1])] += int(line.split()[0])

14848514 b.txt
8504156 c.dat
29116 f
2557 g
62596 h.lst
584 i
4060174 j
8033020 d.log
5626152 d.ext
7214296 k


In [81]:
sizes

defaultdict(int,
            {('/',): 48381165,
             ('/', 'a'): 94853,
             ('/', 'a', 'e'): 584,
             ('/', 'd'): 24933642})

In [82]:
sum(x for x in sizes.values() if x <= 100_000)

95437

In [83]:
min(x for x in sizes.values() if x > 268565)

24933642

In [84]:
932016

932016

In [35]:
g = defaultdict(list)

for arr in brr:
    xrr =  arr.strip().split("\n")
    dirs = xrr[0].split()[-1]
    for item in xrr[:2]:
        isdir, name = item.split()
        if isdir == "dir":
            g[dirs].a
            dirs[name]
    print(xrr)
    break

ValueError: not enough values to unpack (expected 2, got 0)

In [13]:
brr

['',
 ' cd /\n\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n',
 ' cd a\n\ndir e\n29116 f\n2557 g\n62596 h.lst\n',
 ' cd e\n\n584 i\n',
 ' cd ..\n',
 ' cd ..\n',
 ' cd d\n\n4060174 j\n8033020 d.log\n5626152 d.ext\n7214296 k']

In [39]:
# brr = srr.split("\n\n")

# xrr = \
# [
#     [
#         [
#             int(drr) 
#             for drr in crr.split("-")
#         ]
#         for crr in brr.split(",")
#     ]
#     for brr in srr.split("\n")
# ]

# brr = srr.split("\n")

In [245]:
# xrr

In [191]:
res = 0

In [1]:
# sum(sorted(brr)[-3:])

In [ ]:
def test(arr):
    res = 0
    return res